# First clean

Removing duplicate albums based on 1) album name; 2) album id; 3) feature difference

In [2]:
setwd("/home/pasoneto/Documents/github/doc_suomi/code")
source("utils.R")
source("data_cook.R")

`summarise()` regrouping output by 'album_id' (override with `.groups` argument)



In [2]:
new = fread("sample2.csv")
old = fread("grande_sem_low.csv")

base = dplyr::bind_rows(new, old)

In [3]:
replicates = base %>%
    group_by(album_id) %>%
    summarise(overall = sum(valence, energy, loudness, tempo), name = unique(name))

replicates = replicates[order(-replicates$overall), ]

`summarise()` regrouping output by 'album_id' (override with `.groups` argument)



In [4]:
#Calculate 0 differences
valores = c(replicates$overall)
album_ids = c(replicates$album_id)
nome = c(replicates$name)

difs = c()
albums = c()
nomes = c()
nomes2 = c()
for(j in 1:(length(valores)-1)){
    difs[[j]] = abs(valores[j] - valores[j+1])
    albums[[j]] = album_ids[j]
    nomes[[j]] = nome[j]
    nomes2[[j]] = nome[j+1]
    }

replicates = tibble(album_id = albums, difference =difs, nome1 = nomes, nomes2 = nomes2)

In [5]:
nomes_iguais = replicates[order(replicates$difference, replicates$nome1),] %>% filter(nome1 == nomes2)
nomes_iguais = c(nomes_iguais$album_id)

In [6]:
dif_zero = replicates[order(replicates$difference, replicates$nome1),] %>% filter(difference == 0)
dif_zero = c(dif_zero$album_id)

In [7]:
dif_pequena = replicates[order(replicates$difference, replicates$nome1),] %>% filter(difference <= 0.005)
dif_pequena = c(dif_pequena$album_id)

In [8]:
filtro = unique(c(nomes_iguais, dif_zero, dif_pequena))

In [9]:
base %>%
    filter(!album_id %in% filtro) %>% 
    dplyr::select(album_id, track_number, name, valence, energy, loudness, tempo, danceability) %>%
    write.csv("/home/pasoneto/Documents/github/doc_suomi/data/treated_data/data.csv")

# Second clean

Removing albums with wrong number of tracks (from 1 to 10, but missing track 9)

In [24]:
setwd("/home/pasoneto/Documents/github/doc_suomi/data/treated_data")

base = fread("data.csv")
#If album length != largest track_number, remove album 
raw = base %>% group_by(album_id, track_number) %>%
        summarise(valence = mean(valence), energy = mean(energy), 
                  loudness = mean(loudness), tempo = mean(tempo)) %>%
        mutate(album_length = NROW(track_number)) %>% ungroup() %>%
        group_by(album_id) %>%
        filter(max(track_number) != unique(album_length))

albums_out = c(unique(raw$album_id))

base %<>% 
    filter(!album_id %in% albums_out) %>%
    write.csv("/home/pasoneto/Documents/github/doc_suomi/data/treated_data/data.csv")

`summarise()` regrouping output by 'album_id' (override with `.groups` argument)



character(0)

## Third clean

Verifying if there really aren't duplicates.

In [19]:
#There aren't
base() %>% 
    group_by(album_id) %>%
    summarise(valence = sum(valence)) %>% 
    select(album_id, valence) %>% arrange(valence) %>%
    unique()

`summarise()` ungrouping output (override with `.groups` argument)



album_id,valence
<chr>,<dbl>
7zL4yLApULc0HhEpoqOrwG,0.2326
5MOf6d4X7mtvhSwvE5hEkQ,0.2565
2Qnfp8GLuzecgtsqK4JuON,0.2692
1XcSeRzPDXZfgatgAWB1zw,0.3031
7Jm76j0kKzyy2GLExHlVQO,0.3219
3coLBjEk3wxbYrmIKVuSon,0.3260
1u2MIONeSkVJDGC2Ft3efs,0.3483
2jxyIfyUY0yhPANWWrwnca,0.3510
7qpjccYlXA9loqNePT22cy,0.3544


# Cleaning low level

In [3]:
low = fread("/home/pasoneto/Documents/github/doc_suomi/data/raw_dataset/grande_com_low.csv")
low %<>% select(c('danceability', 'energy', 'key', 'loudness_x', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 
'liveness', 'valence', 'tempo_x', 'duration_ms', 'time_signature', 'track_number', 'album_id', 'name', 
'loudness_y', 'tempo_y', 'tempo_confidence', 'key_confidence', 'mode_confidence', 'time_signature_confidence', 'track_id'))

colnames(low) <- c('danceability', 'energy', 'key', 'loudness_overall', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 
'liveness', 'valence', 'tempo_overall', 'duration_ms', 'time_signature', 'track_number', 'album_id', 'name', 
'loudness_continuous', 'tempo_continuous', 'tempo_confidence', 'key_confidence', 'mode_confidence', 'time_signature_confidence', 'track_id')

In [7]:
replicates = low %>%
    group_by(album_id) %>%
    summarise(overall = sum(valence, loudness_overall, energy, tempo_overall), name = unique(name))

replicates = replicates[order(-replicates$overall), ]

#Calculate 0 differences
valores = c(replicates$overall)
album_ids = c(replicates$album_id)
nome = c(replicates$name)

difs = c()
albums = c()
nomes = c()
nomes2 = c()
for(j in 1:(length(valores)-1)){
    difs[[j]] = abs(valores[j] - valores[j+1])
    albums[[j]] = album_ids[j]
    nomes[[j]] = nome[j]
    nomes2[[j]] = nome[j+1]
    }

replicates = tibble(album_id = albums, difference =difs, nome1 = nomes, nomes2 = nomes2)

replicates <- as.data.frame(lapply(replicates, unlist))

nomes_iguais = replicates[order(replicates$difference, replicates$nome1),] %>% filter(nome1 == nomes2)
nomes_iguais = c(nomes_iguais$album_id)

dif_zero = replicates[order(replicates$difference, replicates$nome1),] %>% filter(difference == 0)
dif_zero = c(dif_zero$album_id)

dif_pequena = replicates[order(replicates$difference, replicates$nome1),] %>% filter(difference <= 0.005)
dif_pequena = c(dif_pequena$album_id)

filtro = unique(c(nomes_iguais, dif_zero, dif_pequena))

low %<>%
     filter(!album_id %in% filtro) 

low %>% 
     write.csv("/home/pasoneto/Documents/github/doc_suomi/data/treated_data/data_low.csv")

`summarise()` ungrouping output (override with `.groups` argument)



In [11]:
low %>%
  select(everything()) %>%  # replace to your needs
  summarise_all(oi = sum(is.na(.)))

ERROR: Error in manip_all(.tbl, .funs, enquo(.funs), caller_env(), ..., .caller = "summarise_all"): argument ".funs" is missing, with no default
